In [ ]:
import os.path
from os import path
import numpy as np
import matplotlib.pyplot as pyplot
import gc
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import statistics
import seaborn as sn
import random
import copy
import math
from itertools import permutations

Fill in the following paths that contain:

neutrons_train_path: neutron pulses used for training\
gammas_train_path: gamma pulses used for training\
neutrons_val_path: neutron pulses used for validation\
gammas_val_path: gamma pulses used for validation\
template_n_path: template of the average neutron\
template_g_path: template of the average gamma

Provide the following paths in which values will be saved:

path_model_NP: NoA-DNN of the Deep Detector\
path_model_NP_params: NoA-DNN of the Deep Detector parameters\
path_model_1P: 1st LoA-DNN of the Deep Detector\
path_model_1P_params: 1st LoA-DNN of the Deep Detector parameters\
path_model_2P: 2nd LoA-DNN of the Deep Detector\
path_model_2P_params: 2nd LoA-DNN of the Deep Detector parameters\
path_model_3P: 3rd LoA-DNN of the Deep Detector\
path_model_3P_params: 3rd LoA-DNN of the Deep Detector parameters\
path_model_NP_NM: NoA-DNN of the Deep Detector no margins\
path_model_NP_NM_params: NoA-DNN of the Deep Detector no margins parameters\
path_model_1P_NM: 1st LoA-DNN of the Deep Detector no margins\
path_model_1P_NM_params: 1st LoA-DNN of the Deep Detector no margins parameters\
path_model_2P_NM: 2nd LoA-DNN of the Deep Detector no margins\
path_model_2P_NM_params: 2nd LoA-DNN of the Deep Detector no margins parameters\
path_model_3P_NM: 3rd LoA-DNN of the Deep Detector no margins\
path_model_3P_NM_params: 3rd LoA-DNN of the Deep Detector no margins parameters\
path_corr_trig_params: NCC parameters\
detector_config_path: constants are saved here


In [ ]:
# Parameters:
arrival_rate = 6e6
g_n_ratio = 1

# Constants:
WL_trigger_middle = 140 # length in samples of the window w_t analyzed by the Deep Detector
margin = 100 # length in samples of the margins used by the Deep Detector
num_data = 40000 # Number of windows used for training
starting_index = 0
template_length_orig_features = 400
Ts = 1e-9
num_original_features = 13
num_classification_features = 8
num_attention_features = 8
new_starting_point = 20
pulse_len_after_starting = 1000
time_offset = 15
success_marg = 15
L_deriv_orig_features = 500
delta_orig_features = 8
short_gate_class_features = 30
short_gate_orig_features = 30
long_gate_class_features = 100
long_gate_orig_features = 100
max_random_offset = 15
cont_starting_time = 10000
original_pulse_len = 9460
std_noise_total = 0.00259578
std_noise_curr_constant = 0.000269
noise_mean = 0
low_NN = 15
high_NN = 15
len_true_inf = num_original_features + 9
WL_trigger = int(WL_trigger_middle + 2*margin)
lambda_n = int(arrival_rate/(1+g_n_ratio))
lambda_g = int((arrival_rate*g_n_ratio)/(1+g_n_ratio))
std_noise_curr = std_noise_curr_constant*np.sqrt(original_pulse_len*arrival_rate*Ts)
std_added_noise = np.sqrt(np.power(std_noise_total,2) - np.power(std_noise_curr,2))
jump = WL_trigger_middle

run <your-path> 'General_Functions.ipynb>
run <your-path> 'Deep_Detector_Functions.ipynb>

neutrons_train = np.load(neutrons_train_path)
gammas_train = np.load(gammas_train_path)
neutrons_val = np.load(neutrons_val_path)
gammas_val = np.load(gammas_val_path)
template_n = np.load(template_n_path)
template_g = np.load(template_g_path)

cont_data,cont_labels,true_inf,cont_peak_labels,true_inf_peaks,last_arrival_time = generate_pulse_train_peak_labels(neutrons_train,gammas_train,lambda_n,
                                                                                      lambda_g, add_noise = True)

train_data_0,train_labels_0,train_data_1,train_labels_1,train_arr_times_1,train_data_2,train_labels_2,train_arr_times_2,train_data_3,train_labels_3,train_arr_times_3 = create_data_for_training_deep_trigger_new('train',neutrons_train,gammas_train)
train_data_model_NP, train_labels_model_NP = mix_data_for_model_num_pulses(train_data_0,train_data_1,train_data_2,train_data_3,train_labels_0,train_labels_1,train_labels_2,train_labels_3)

val_data_0,val_labels_0,val_data_1,val_labels_1,val_arr_times_1,val_data_2,val_labels_2,val_arr_times_2,val_data_3,val_labels_3,val_arr_times_3 = create_data_for_training_deep_trigger_new('val',neutrons_val,gammas_val)
val_data_model_NP, val_labels_model_NP = mix_data_for_model_num_pulses(val_data_0,val_data_1,val_data_2,val_data_3,val_labels_0,val_labels_1,val_labels_2,val_labels_3)

# Deep Detector Layers
model_NP_layer1 = 150
model_NP_layer2 = 60
model_NP_layer3 = 25
model_NP_layer4 = 10
model_NP_layer5 = 4
model_P3_layer5 = 3
model_P2_layer5 = 5
model_P2_layer6 = 2
model_P1_layer5 = 3
model_P1_layer6 = 1
# Additional Deep Detector without margin layers
model_No_Margin_layer1 = 100
model_No_Margin_layer2 = 50

run <your-path> 'Deep_Detector_Networks.ipynb>
run <your-path> 'Deep_Detector_No_Margins_Networks.ipynb>

## Training the Deep Detector

## Training the NoA-DNN
model_NP_lr = 1e-3
model_NP_bs = 30
model_NP_epochs = 40
run_Model_Trigger_Classification(train_labels_model_NP,train_data_model_NP,
                                val_labels_model_NP,val_data_model_NP,val_labels_model_NP,
                                val_data_model_NP,path_model_NP,path_model_NP_params,
                                model_NP_lr,model_NP_bs,model_NP_epochs)

## Training the 1st LoA-DNN
model_1P_lr = 1e-3
model_1P_bs = 30
model_1P_epochs = 500
run_Model_Arrival_Times_1(train_arr_times_1,train_data_1,
                                val_arr_times_1,val_data_1,val_arr_times_1,val_data_1,
                                path_model_1P,path_model_1P_params,
                                model_1P_lr,model_1P_bs,model_1P_epochs)

## Training the 2nd LoA-DNN
model_2P_lr = 1e-3
model_2P_bs = 30
model_2P_epochs = 500
run_Model_Arrival_Times_2(train_arr_times_2,train_data_2,
                                val_arr_times_2,val_data_2,val_arr_times_2,val_data_2,
                                path_model_2P,path_model_2P_params,
                                model_2P_lr,model_2P_bs,model_2P_epochs)

## Training the 3rd LoA-DNN
model_3P_lr = 1e-3
model_3P_bs = 30
model_3P_epochs = 500
run_Model_Arrival_Times_3(train_arr_times_3,train_data_3,
                                val_arr_times_3,val_data_3,val_arr_times_3,val_data_3,
                                path_model_3P,path_model_3P_params,
                                model_3P_lr,model_3P_bs,model_3P_epochs)


## Training the Deep Detector without the use of margins

## Training the NoA-DNN
run_Model_Trigger_Classification_NM(train_labels_model_NP,train_data_model_NP[:,margin:margin + WL_trigger_middle],
                                val_labels_model_NP,val_data_model_NP[:,margin:margin + WL_trigger_middle],val_labels_model_NP,
                                val_data_model_NP[:,margin:margin + WL_trigger_middle],path_model_NP_NM,path_model_NP_NM_params,
                                model_NP_lr,model_NP_bs,model_NP_epochs)

## Training the 1st LoA-DNN
run_Model_Arrival_Times_1_NM(train_arr_times_1,train_data_1[:,margin:margin + WL_trigger_middle],
                                val_arr_times_1,val_data_1[:,margin:margin + WL_trigger_middle],val_arr_times_1,val_data_1[:,margin:margin + WL_trigger_middle],
                                path_model_1P_NM,path_model_1P_NM_params,
                                model_1P_lr,model_1P_bs,model_1P_epochs)

## Training the 2nd LoA-DNN
run_Model_Arrival_Times_2_NM(train_arr_times_2,train_data_2[:,margin:margin + WL_trigger_middle],
                                val_arr_times_2,val_data_2[:,margin:margin + WL_trigger_middle],val_arr_times_2,val_data_2[:,margin:margin + WL_trigger_middle],
                                path_model_2P_NM,path_model_2P_NM_params,
                                model_2P_lr,model_2P_bs,model_2P_epochs)

## Training the 3rd LoA-DNN
run_Model_Arrival_Times_3_NM(train_arr_times_3,train_data_3[:,margin:margin + WL_trigger_middle],
                                val_arr_times_3,val_data_3[:,margin:margin + WL_trigger_middle],val_arr_times_3,val_data_3[:,margin:margin + WL_trigger_middle],
                                path_model_3P_NM,path_model_3P_NM_params,
                                model_3P_lr,model_3P_bs,model_3P_epochs)


# Optimizing Normalized Cross-Correlation detector
param1_lst = np.arange(1,30,1) # Correlation starting index
param2_lst = np.arange(30,200,1) # Correlation ending index
param3_lst = np.linspace(0.5,1,100) # Detection Threshold
param4_lst = np.arange(1,200,1) # Minimum distance between detections
param5_lst = np.arange(1,80,1) # Pre-trigger

best_F1 = -100
best_param1, best_param2, best_param3, best_param4, best_param5 = 0,0,0,0,0

for par1 in param1_lst:
  for par2 in param2_lst:
    cont_corr_sig1 = generate_correlation_data(cont_data,template_g,par1,par2)
    cont_corr_sig2 = generate_correlation_data(cont_data,template_n,par1,par2)
    for par3 in param3_lst:
      for par4 in param4_lst:
        corr_labels = create_Corr_THR_labels(cont_corr_sig1,cont_corr_sig2,last_arrival_time,par3,par4)
        for par5 in param5_lst:
          all_counts,MD,FA,precision,recall,F1 = calc_count_information(cont_labels,corr_labels,last_arrival_time,par5,(low_NN + high_NN) - par5)
          if F1 > best_F1:
            best_F1,best_param1,best_param2,best_param3,best_param4,best_param5 = F1,par1,par2,par3,par4,par5

corr_trig_params = [best_param1,best_param2,best_param3,best_param4,best_param5]

np.save(path_corr_trig_params,corr_trig_params)

constants_dict = {
    "arrival_rate": arrival_rate,
    "g_n_ratio": g_n_ratio,
    "WL_trigger_middle": WL_trigger_middle,
    "margin": margin,
    "num_data": num_data,
    "starting_index": starting_index,
    "template_length_orig_features": template_length_orig_features,
    "Ts": Ts,
    "num_original_features": num_original_features,
    "num_classification_features": num_classification_features,
    "num_attention_features": num_attention_features,
    "new_starting_point": new_starting_point,
    "pulse_len_after_starting": pulse_len_after_starting,
    "time_offset": time_offset,
    "success_marg": success_marg,
    "L_deriv_orig_features": L_deriv_orig_features,
    "delta_orig_features": delta_orig_features,
    "short_gate_class_features": short_gate_class_features,
    "short_gate_orig_features": short_gate_orig_features,
    "long_gate_class_features": long_gate_class_features,
    "long_gate_orig_features": long_gate_orig_features,
    "max_random_offset": max_random_offset,
    "cont_starting_time": cont_starting_time,
    "original_pulse_len": original_pulse_len,
    "std_noise_total": std_noise_total,
    "std_noise_curr_constant": std_noise_curr_constant,
    "noise_mean": noise_mean,
    "low_NN": low_NN,
    "high_NN": high_NN,
    "len_true_inf": len_true_inf,
    "WL_trigger": WL_trigger,
    "lambda_n": lambda_n,
    "lambda_g": lambda_g,
    "std_noise_curr": std_noise_curr,
    "std_added_noise": std_added_noise,
    "jump": jump,
    "arrival_rate_text": arrival_rate_text,
    "curr_path": curr_path,
    "model_NP_layer1": model_NP_layer1,
    "model_NP_layer2": model_NP_layer2,
    "model_NP_layer3": model_NP_layer3,
    "model_NP_layer4": model_NP_layer4,
    "model_NP_layer5": model_NP_layer5,
    "model_P3_layer5": model_P3_layer5,
    "model_P2_layer5": model_P2_layer5,
    "model_P2_layer6": model_P2_layer6,
    "model_P1_layer5": model_P1_layer5,
    "model_P1_layer6": model_P1_layer6,
    "model_NP_lr": model_NP_lr,
    "model_NP_bs": model_NP_bs,
    "model_NP_epochs": model_NP_epochs,
    "model_3P_lr": model_3P_lr,
    "model_3P_bs": model_3P_bs,
    "model_3P_epochs": model_3P_epochs,
    "model_2P_lr": model_2P_lr,
    "model_2P_bs": model_2P_bs,
    "model_2P_epochs": model_2P_epochs,
    "model_1P_lr": model_1P_lr,
    "model_1P_bs": model_1P_bs,
    "model_1P_epochs": model_1P_epochs,

    "model_No_Margin_layer1": model_No_Margin_layer1,
    "model_No_Margin_layer2": model_No_Margin_layer2,
}

with open(detector_config_path, "wb") as file:
    pickle.dump(constants_dict, file)